In [108]:
import urllib
import pymorphy2
import pandas as pd
import openpyxl

from bs4 import BeautifulSoup

In [92]:
base_url = 'http://www.oktmo.ru/locality_type_registry/'

page = urllib.request.urlopen(base_url)
soup = BeautifulSoup(page)

type_cities = soup.find_all('p')
for i, elem in enumerate(type_cities):
    if ' - поиск по коду' in elem.text:
        type_cities = type_cities[i + 1:]
        break
type_cities = [elem.contents[2][2:-1] for elem in type_cities]
type_cities = sorted(type_cities, key=len, reverse=True)

In [93]:
base_url = 'http://www.oktmo.ru/locality_registry'

page = urllib.request.urlopen(base_url)
soup = BeautifulSoup(page)

pages_url = soup.find_all('p')
for i, elem in enumerate(pages_url):
    if ' - поиск по коду' in elem.text:
        pages_url = pages_url[i + 1:]
        break
pages_url = [elem.contents for elem in pages_url]

# morph = pymorphy2.MorphAnalyzer()
for i, elem in enumerate(pages_url):
    if ' Населенные пункты, входящие в состав муниципальных образований ' in pages_url[i][1]:
        pages_url[i][1] = pages_url[i][1].replace(' Населенные пункты, входящие в состав муниципальных образований ', '')
    elif ' Муниципальные районы и городские округа ' in pages_url[i][1]:
        pages_url[i][1] = pages_url[i][1].replace(' Муниципальные районы и городские округа ', '')
    
#     normalized_name = []
#     for word in pages_url[i][1].split():
#         parsed_token = morph.parse(word)
#         normal_form = parsed_token[0].normal_form
#         normalized_name.append(normal_form)
        
#     pages_url[i][1] = ' '.join(normalized_name)
    
data_res = []

for page_url, text in pages_url:
    page = urllib.request.urlopen(page_url.a['href'])
    soup = BeautifulSoup(page)
    
    data = soup.find_all('p')
    for i, elem in enumerate(data):
        if 'Найдено: ' in elem.text:
            data = data[i + 1:]
            break
    for elem in data:
        code = elem.text[:14]
        city = elem.text[16:]
        for type_city in type_cities:
            if type_city in city:
                city_type = type_city
                city_name = city[len(city_type) + 1:]
                break
        else:
            type_city = 'не найден'
            name_city = city
        data_res += [[code, city_type, city_name, text]]

In [102]:
# for elem in data_res:
#     if len(elem) != 4:
#         print(elem)
df = pd.DataFrame(data_res, columns=['Код', 'Тип населенного пункта', 'Наименование', 'Субъект РФ'])
df = df[['Тип населенного пункта', 'Наименование', 'Субъект РФ']].drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123920 entries, 0 to 156367
Data columns (total 3 columns):
Тип населенного пункта    123920 non-null object
Наименование              123920 non-null object
Субъект РФ                123920 non-null object
dtypes: object(3)
memory usage: 3.8+ MB


In [107]:
# df.to_csv('Регистр населенных пунктов РФ.xlsx', sep=';', encoding='utf8', index=False)

In [114]:
file_name = 'Регистр населенных пунктов РФ.xlsx'
sheet_name = 'Sheet1'

work_book = openpyxl.Workbook()
sheet = work_book.active
sheet.title = sheet_name
work_book.save(file_name)

work_book = openpyxl.load_workbook(file_name)
sheet = work_book[sheet_name]

with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', datetime_format='DD.MM.YYYY') as writer:
    writer.book = work_book
    writer.sheets = dict((ws.title, ws) for ws in work_book.worksheets)
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    
work_book.save(file_name)